In [1]:
import pandas as pd
import numpy as np

# reading assumptions
assumptions_df = pd.read_csv('data/assumptions.csv')

# accessing mortality rates
male_mortality_rates = pd.read_csv("data/male_mortality_rates.csv")
female_mortality_rates = pd.read_csv("data/female_mortality_rates.csv")

person_age = int(input("Enter the person's age (25-65 yrs): "))
person_gender = input("Person's Gender (M or F): ")
retirement_age = 45
term_length = retirement_age - person_age

Enter the person's age (25-65 yrs):  34
Person's Gender (M or F):  M


In [2]:
def vlookup(value,df,coli,colo):
    """ 
    vlookup function to reference other tables
    """
    return next(iter(df.loc[df[coli]==value][colo]), None)

In [5]:
def create_male_decrements_table():
    "function to create the male decrements table"
    
    male_decrements_table = []

    age = person_age
    
    for i in range(term_length):
        qxd = vlookup(age,male_mortality_rates,'age','qxd')
        qxr = vlookup(age,male_mortality_rates,'age','qxr')
        aqxd = qxd * (1 - 0.5 * qxr)
        aqxr = qxr * (1 - 0.5 * qxd)
        apx = 1 - (aqxd + aqxr)
        
        if i == 0:
            t_1apx = 1
        else:
            t_1apx = (male_decrements_table[i-1]['apx'])*(male_decrements_table[i-1]['t_1apx'])
        
        male_decrements_table.append({
            'age': age,
            'qxd': qxd,
            'qxr': qxr,
            'aqxd': aqxd,
            'aqxr': aqxr,
            'apx': apx,
            't_1apx': t_1apx
        })
        age = age + 1
            
    return pd.DataFrame(male_decrements_table) 

In [6]:
def create_female_decrements_table():
    "function to create the female decrements table"
    
    female_decrements_table = []
    
    #get initial age
    age =  female_mortality_rates.age[0]
    
    for i in range(21):
        qxd = female_mortality_rates.qxd[i]
        qxr = female_mortality_rates.qxr[i]
        aqxd = qxd * (1 - 0.5 * qxr)
        aqxr = qxr * (1 - 0.5 * qxd)
        apx = 1 - (aqxd + aqxr)
        
        if i == 0:
            t_1apx = 1
        else:
            t_1apx = (female_decrements_table[i-1]['apx'])*(female_decrements_table[i-1]['t_1apx'])
        
        female_decrements_table.append({
            'age': age,
            'qxd': qxd,
            'qxr': qxr,
            'aqxd': aqxd,
            'aqxr': aqxr,
            'apx': apx,
            't_1apx': t_1apx
        })
        age = age + 1
            
    return pd.DataFrame(female_decrements_table) 

In [7]:
#create decrements table based on person's gender
if person_gender == "M":
    decrements_table = create_male_decrements_table()
else:
    decrements_table = create_female_decrements_table()

In [8]:
male_pricing_defaults = pd.read_csv("data/male_pricing_defaults.csv")

In [12]:
def create_table_with_qx_lx_dx():
    "function to calculate dx and lx"
    
    qx_lx_dx = []
    age = person_age
    
    for i in range(term_length):
        qx = vlookup(age,male_pricing_defaults,'age','qx')
        
        if i == 0:
            lx = 100000
        else:
            lx = qx_lx_dx[i-1]['lx'] - qx_lx_dx[i-1]['dx']
        
        dx = qx * lx
        
        qx_lx_dx.append({
            'age': age,
            'qx': qx,
            'lx': lx,
            'dx': dx
        })
        age = age + 1
    return pd.DataFrame(qx_lx_dx)

In [13]:
table_with_qx_lx_dx = create_table_with_qx_lx_dx()
table_with_qx_lx_dx

,age,qx,lx,dx
0,34,0.002246,100000.000000,224.600000
1,35,0.002297,99775.400000,229.184094
2,36,0.002390,99546.215906,237.915456
3,37,0.002518,99308.300450,250.058301
4,38,0.002674,99058.242150,264.881740
5,39,0.002855,98793.360410,282.055044
6,40,0.003049,98511.305366,300.360970
7,41,0.003253,98210.944396,319.480202
8,42,0.003458,97891.464194,338.508683
9,43,0.003656,97552.955511,356.653605


In [14]:
pricing_interest = 0.06
sum_assured = 10000000
V = 1/(1+pricing_interest)
premium = 1470259.9692

In [17]:
def calculate_benefit_and_claim_expenses():
    "function to calculate benefit and claims expenses"
    
    benefit_and_claims = []
    
    for i in range(35):
        time = i + 1
        discount = V ** time
        male_probability = male_pricing_defaults.probability[i]
        EPV = sum_assured * discount * male_probability
        
        benefit_and_claims.append({
            "time t": time,
            "amount": sum_assured,
            "discount": discount,
            "probability": male_probability,
            "EPV": EPV
        })
        
    last_probability = table_with_qx_lx_dx.lx.iat[-1] / table_with_qx_lx_dx.lx[0]
    benefit_and_claims.append({
            "time t": time,
            "amount": sum_assured,
            "discount": discount,
            "probability": last_probability,
            "EPV": sum_assured * discount * last_probability
        })
    return pd.DataFrame(benefit_and_claims)

In [18]:
benefit_and_claim_expenses = calculate_benefit_and_claim_expenses()

In [19]:
# EPV of benefits and claims
EPV_benefits = round(sum(benefit_and_claim_expenses.EPV), 2)

In [20]:
def calculate_initial_and_regular_expenses():
    "function to calculate initial and regular expenses"
    
    initial_and_regular_expenses = []
    
    for i in range(term_length):
        if i == 0:
            amount = 0.4 * premium
        else:
            amount = 0.05 * premium
            
        discount = V ** i
        probability = table_with_qx_lx_dx.lx[i] / table_with_qx_lx_dx.lx[0]
        EPV = amount * discount * probability
        
        initial_and_regular_expenses.append({
            "time t": i,
            "amount": amount,
            "discount": discount,
            "probability": probability,
            "EPV": EPV
        })
        
    return pd.DataFrame(initial_and_regular_expenses)

In [21]:
initial_and_regular_expenses = calculate_initial_and_regular_expenses()

In [22]:
# EPV of initial and regular expenses
EPV_expenses = round(sum(initial_and_regular_expenses.EPV), 2)

In [23]:
def calculate_initial_and_renewal_commission():
    "function to calculate initial and renewal commission"
    
    initial_and_renewal_commission = []
    
    for i in range(term_length):
        if i == 0:
            amount = 0.4 * premium
        else:
            amount = 0.08 * premium
            
        discount = V ** i
        probability = table_with_qx_lx_dx.lx[i] / table_with_qx_lx_dx.lx[0]
        EPV = amount * discount * probability
        
        initial_and_renewal_commission.append({
            "time t": i,
            "amount": amount,
            "discount": discount,
            "probability": probability,
            "EPV": EPV
        })
        
    return pd.DataFrame(initial_and_renewal_commission)

In [24]:
initial_and_renewal_commission = calculate_initial_and_renewal_commission()

In [25]:
#EPV of initial and renewal commission
EPV_commission = round(sum(initial_and_renewal_commission.EPV), 2)

In [27]:
def calculate_premiums():
    "function to calculate premiums"
    
    premiums = []
    
    for i in range(term_length):            
        discount = V ** i
        probability = table_with_qx_lx_dx.lx[i] / table_with_qx_lx_dx.lx[0]
        EPV = premium * discount * probability
        
        premiums.append({
            "time t": i,
            "amount": premium,
            "discount": discount,
            "probability": probability,
            "EPV": EPV
        })
        
    return pd.DataFrame(premiums)

In [28]:
calculated_premiums = calculate_premiums()

In [38]:
table_with_qx_lx_dx

,age,qx,lx,dx
0,34,0.002246,100000.000000,224.600000
1,35,0.002297,99775.400000,229.184094
2,36,0.002390,99546.215906,237.915456
3,37,0.002518,99308.300450,250.058301
4,38,0.002674,99058.242150,264.881740
5,39,0.002855,98793.360410,282.055044
6,40,0.003049,98511.305366,300.360970
7,41,0.003253,98210.944396,319.480202
8,42,0.003458,97891.464194,338.508683
9,43,0.003656,97552.955511,356.653605


In [29]:
#EPV of premiums
EPV_premiums = round(sum(calculated_premiums.EPV), 2)

In [56]:
def calculate_other_benefits():
    "function to calculate other benefits"
    
    other_benefits = []
    
    for i in range(term_length):
        time = i + 1
        lookup_value = person_age + time
        get_probability = vlookup(lookup_value, table_with_qx_lx_dx, 'age', 'qx')
        if get_probability is not None:
            amount = 0.8 * premium
            discount = V ** time
            probability = get_probability
        else:
            amount = 0
            discount = 0
            probability = 0
        EPV = amount * discount * probability
        
        other_benefits.append({
            "time t": time,
            "amount": amount,
            "discount": discount,
            "probability": probability,
            "EPV": EPV
        })
        
    return pd.DataFrame(other_benefits)

In [58]:
other_benefits = calculate_other_benefits()

In [59]:
#EPV of other benefits
EPV_other_benefits = round(sum(other_benefits.EPV), 2)

In [60]:
unit_fund_growth_rate = float(assumptions_df[assumptions_df.criteria == 'unit fund growth rate'].value)
nonunit_fund_growth_rate = float(assumptions_df[assumptions_df.criteria == 'non-unit fund growth rate'].value)
risk_discount_rate = float(assumptions_df[assumptions_df.criteria == 'risk discount rate'].value)
mgt_charge = float(assumptions_df[assumptions_df.criteria == 'mgt charge'].value)
bid_offer_spread = float(assumptions_df[assumptions_df.criteria == 'bid-offer spread'].value)

In [61]:
def calculate_unit_fund():
    "function to calculate unit fund"
    
    unit_fund = []
    
    for i in range(5):
        year = i + 1
        premium_allocated = 0.85 * premium
        cost_of_allocation = premium_allocated * (1 - bid_offer_spread)
        
        if i == 0:
            fund_after_allocation = cost_of_allocation
        else:
            fund_after_allocation = unit_fund[i-1]['fund at year end'] + cost_of_allocation
                                                   
        fund_before_mgt_charge = fund_after_allocation * (1 + unit_fund_growth_rate)
        management_charge = fund_before_mgt_charge * mgt_charge
        fund_at_year_end = fund_before_mgt_charge - management_charge
                                                   
        unit_fund.append({
            'year': year,
            'premium received': premium,
            'premium allocated': premium_allocated,
            'cost of allocation': cost_of_allocation,
            'fund after allocation': fund_after_allocation,
            'fund before mgt charge': fund_before_mgt_charge,
            'mgt charge': management_charge,
            'fund at year end': fund_at_year_end
        })
    
    return pd.DataFrame(unit_fund)

In [64]:
unit_fund = calculate_unit_fund()

In [67]:
def calculate_non_unit_fund():
    "function to calculate non unit fund"
    
    non_unit_fund = []
    inflation_rate = float(assumptions_df[assumptions_df.criteria == 'inflation rate'].value)
    
    for i in range(5):
        year = i + 1
        premium_less_cost_of_allocation = unit_fund['premium received'][i] - unit_fund['cost of allocation'][i]
        if (year == 1):
            expenses = -0.4 * unit_fund['premium received'][i]
        elif (year == 2):
            expenses = -0.05 * unit_fund['premium received'][i]
        elif (year == 3):
            expenses = -0.05 * unit_fund['premium received'][i] * (1 + inflation_rate)
        else:
            expenses = non_unit_fund[i-1]['expenses'] * (1 + inflation_rate)
        
        interest = (premium_less_cost_of_allocation + expenses) * nonunit_fund_growth_rate
        extra_death_cost = -(sum_assured - unit_fund['fund at year end'][i]) * decrements_table.aqxd[i]
        mgt_charge = unit_fund['mgt charge'][i]
        profit = premium_less_cost_of_allocation + expenses + interest + extra_death_cost + mgt_charge
        
        non_unit_fund.append({
            'year': year,
            'premium less cost of allocation': premium_less_cost_of_allocation,
            'expenses': expenses,
            'interest': interest,
            'extra death cost': extra_death_cost,
            'mgt charge': mgt_charge,
            'profit': profit        
        })
        
    return pd.DataFrame(non_unit_fund)
            

In [68]:
non_unit_fund = calculate_non_unit_fund()

In [71]:
def calculate_pv_profit():
    "function to calculate the present value of profit"
    
    pv_profit = []
    
    for i in range(5):
        year = i + 1
        profit_in_year_t = non_unit_fund['profit'][i]
        t_1apx = decrements_table.t_1apx[i]
        profit_signature = profit_in_year_t * t_1apx
        discount_factor = 1/((1+risk_discount_rate)**year)
        discounted_profit = profit_signature * discount_factor
        
        pv_profit.append({
            'year': year,
            'profit in year t': profit_in_year_t,
            't-1(ap)x': t_1apx,
            'profit signature': profit_signature,
            'discount factor': discount_factor,
            'discounted profit': discounted_profit
        })
        
    return pd.DataFrame(pv_profit)

In [72]:
pv_profit = calculate_pv_profit()

In [73]:
sum_pv_profit = sum(pv_profit['discounted profit'])

In [76]:
def calculate_pv_premiums():
    "function to calculate present value of premiums"
    
    pv_premiums = []
    
    for i in range(5):
        year = i + 1
        t_1apx = decrements_table.t_1apx[i]
        discount_factor_v = 1/((1+risk_discount_rate)**(year-1))
        discounted_premium = premium * t_1apx * discount_factor_v
        
        pv_premiums.append({
            'year': year,
            'premium': premium,
            't-1(ap)x': t_1apx,
            'discount factor v^(t-1)': discount_factor_v,
            'discounted premium': discounted_premium
        })
        
    return pd.DataFrame(pv_premiums)        

In [77]:
pv_premiums = calculate_pv_premiums()

In [78]:
sum_pv_premiums = sum(pv_premiums['discounted premium'])

In [79]:
premium_rate = (premium/sum_assured) * 100

In [80]:
proft_margin = (sum_pv_profit/sum_pv_premiums) * 100

In [81]:
print("Premium Rate:: ", premium_rate)
print("Profit Margin: ",proft_margin)

Premium Rate::  14.702599691999998
Profit Margin:  2.42801325118567
